<a href="https://colab.research.google.com/github/snakeeyes021/pneumonia-classification/blob/Madoria/Copy_of_Madoria_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Madoria 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from sklearn.preprocessing import StandardScaler, LabelBinarizer

import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPool2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
from sklearn.metrics import confusion_matrix
import PIL
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# pip install keras-visualizer
# !apt-get -qq install -y graphviz && pip install graphviz

Starting off with the content from the seed notebook. 

In [ ]:
train_dir= "/content/gdrive/MyDrive/chest_xray/train"
test_dir= "/content/gdrive/MyDrive/chest_xray/test"
val_dir= "/content/gdrive/MyDrive/chest_xray/val"

In [ ]:
train_norm_dir = os.path.join(train_dir, 'NORMAL')
train_pneu_dir = os.path.join(train_dir, 'PNEUMONIA')
val_norm_dir = os.path.join(val_dir, 'NORMAL')
val_pneu_dir = os.path.join(val_dir, 'PNEUMONIA')
test_norm_dir = os.path.join(test_dir, 'NORMAL')
test_pneu_dir = os.path.join(test_dir, 'PNEUMONIA')
train_norm_dir

'/content/gdrive/MyDrive/chest_xray/train/NORMAL'

In [ ]:
len(os.listdir(train_norm_dir))

1221

In [ ]:
len(os.listdir(train_pneu_dir))

3755

In [ ]:
def image_paths(filepath):
    filenames = os.listdir(filepath)
    filepaths = [os.path.join(filepath, name) for name in filenames]
    return [img for img in filepaths]


train_norm_images = image_paths(train_norm_dir)
train_pneu_images = image_paths(train_pneu_dir)

In [ ]:
# Normal example
PIL.Image.open(train_norm_images[0])

In [ ]:
test_generator = ImageDataGenerator(rotation_range=30, zoom_range= 0.2, width_shift_range=0.3, 
                                   height_shift_range=0.3, shear_range= 0.2, rescale=1./255).flow_from_directory(
        test_dir, batch_size=624,
        target_size=(64, 64)) 

val_generator = ImageDataGenerator(rotation_range=30, zoom_range= 0.2, width_shift_range=0.3, 
                                   height_shift_range=0.3, shear_range= 0.2, rescale=1./255).flow_from_directory(
        val_dir, batch_size=256,
        target_size=(64, 64))

train_generator = ImageDataGenerator(rotation_range=30, zoom_range= 0.2, width_shift_range=0.3, 
                                   height_shift_range=0.3, shear_range= 0.2, rescale=1./255).flow_from_directory(
        train_dir, batch_size=4976,  
        target_size=(64, 64))

Found 624 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Found 4976 images belonging to 2 classes.


In [ ]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [ ]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(4976, 12288)
(624, 12288)
(256, 12288)


In [ ]:
train_y = np.reshape(train_labels[:,0], (4976,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (256,1))

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [ ]:
batch = 8

history = model.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=batch, verbose= 1,
                    validation_data=(val_images, val_y))

results_train = model.evaluate(train_images, train_y)
results_test = model.evaluate(test_images, test_y)

print(f'This is the training result: {results_train}')
print(f'This is the test results: {results_test}')

Epoch 1/50
622/622 [==============================] - 67s 107ms/step - loss: 0.5054 - acc: 0.7520 - val_loss: 0.8595 - val_acc: 0.5039
Epoch 2/50
622/622 [==============================] - 35s 56ms/step - loss: 0.4530 - acc: 0.7653 - val_loss: 0.7141 - val_acc: 0.6094
Epoch 3/50
622/622 [==============================] - 35s 56ms/step - loss: 0.4002 - acc: 0.8069 - val_loss: 0.5379 - val_acc: 0.7227
Epoch 4/50
622/622 [==============================] - 35s 56ms/step - loss: 0.3338 - acc: 0.8404 - val_loss: 0.4305 - val_acc: 0.7969
Epoch 5/50
622/622 [==============================] - 35s 57ms/step - loss: 0.2917 - acc: 0.8708 - val_loss: 0.4314 - val_acc: 0.8047
Epoch 6/50
622/622 [==============================] - 35s 55ms/step - loss: 0.2628 - acc: 0.8861 - val_loss: 0.6790 - val_acc: 0.7617
Epoch 7/50
622/622 [==============================] - 35s 56ms/step - loss: 0.2328 - acc: 0.8999 - val_loss: 0.3761 - val_acc: 0.8516
Epoch 8/50
622/622 [==============================] - 35s 56m

In [ ]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3), kernel_regularizer=regularizers.l2(0.005)))
model2.add(layers.Dropout(0.3))

model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(32, (4, 4), activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model2.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.005)))

model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [ ]:
history = model2.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=batch, verbose=1,
                    validation_data=(val_images, val_y))

results_train = model2.evaluate(train_images, train_y)
results_test = model2.evaluate(test_images, test_y)

print(f'This is the training result: {results_train}')
print(f'This is the test results: {results_test}')

Epoch 1/50
622/622 [==============================] - 45s 72ms/step - loss: 0.6134 - acc: 0.7546 - val_loss: 0.7775 - val_acc: 0.5000
Epoch 2/50
622/622 [==============================] - 45s 72ms/step - loss: 0.5509 - acc: 0.7546 - val_loss: 0.8206 - val_acc: 0.5000
Epoch 3/50
622/622 [==============================] - 45s 72ms/step - loss: 0.5189 - acc: 0.7546 - val_loss: 0.8343 - val_acc: 0.5000
Epoch 4/50
622/622 [==============================] - 44s 71ms/step - loss: 0.5111 - acc: 0.7546 - val_loss: 0.7234 - val_acc: 0.5000
Epoch 5/50
622/622 [==============================] - 44s 71ms/step - loss: 0.5075 - acc: 0.7546 - val_loss: 0.7429 - val_acc: 0.5000
Epoch 6/50
622/622 [==============================] - 44s 70ms/step - loss: 0.5048 - acc: 0.7546 - val_loss: 0.7958 - val_acc: 0.5000
Epoch 7/50
622/622 [==============================] - 46s 73ms/step - loss: 0.5054 - acc: 0.7546 - val_loss: 0.7607 - val_acc: 0.5000
Epoch 8/50
622/622 [==============================] - 46s 74ms

In [ ]:
normal= len(os.listdir(train_norm_dir))
pneumonia= len(os.listdir(train_pneu_dir))

train_count = normal + pneumonia
weight_for_0 = (1 / normal) * (train_count) / 2.0
weight_for_1 = (1 / pneumonia) * (train_count) / 2.0

weight = {0: weight_for_0, 1: weight_for_1}

print("Weight for class 0: {:.2f}".format(weight_for_0))
print("Weight for class 1: {:.2f}".format(weight_for_1))

Weight for class 0: 2.04
Weight for class 1: 0.66


In [ ]:
model3 = models.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3), kernel_regularizer=regularizers.l2(0.005)))
model3.add(layers.Dropout(0.3))

model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(32, (4, 4), activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model3.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.005)))

model3.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [ ]:
history = model3.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=batch, verbose=1, class_weight=weight,
                    validation_data=(val_images, val_y))

results_train = model3.evaluate(train_images, train_y)
results_test = model3.evaluate(test_images, test_y)

print(f'This is the training result: {results_train}')
print(f'This is the test results: {results_test}')

Epoch 1/50
622/622 [==============================] - 46s 74ms/step - loss: 0.5337 - acc: 0.7546 - val_loss: 1.1824 - val_acc: 0.5000
Epoch 2/50
622/622 [==============================] - 46s 74ms/step - loss: 0.5224 - acc: 0.7546 - val_loss: 1.1586 - val_acc: 0.5000
Epoch 3/50
622/622 [==============================] - 47s 75ms/step - loss: 0.5190 - acc: 0.7546 - val_loss: 1.0781 - val_acc: 0.5000
Epoch 4/50
622/622 [==============================] - 46s 75ms/step - loss: 0.5124 - acc: 0.7546 - val_loss: 1.4135 - val_acc: 0.5000
Epoch 5/50
622/622 [==============================] - 46s 74ms/step - loss: 0.5155 - acc: 0.7546 - val_loss: 1.2127 - val_acc: 0.5000
Epoch 6/50
622/622 [==============================] - 50s 80ms/step - loss: 0.5111 - acc: 0.7546 - val_loss: 1.3484 - val_acc: 0.5000
Epoch 7/50
622/622 [==============================] - 48s 77ms/step - loss: 0.5089 - acc: 0.7546 - val_loss: 1.0778 - val_acc: 0.5000
Epoch 8/50
622/622 [==============================] - 48s 77ms

In [ ]:
model4 = models.Sequential()
model4.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))
model4.add(layers.Dropout(0.3))

model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Conv2D(32, (4, 4), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Flatten())
model4.add(layers.Dense(64, activation='relu'))
model4.add(layers.Dense(1, activation='sigmoid'))

model4.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [ ]:
history = model4.fit(train_images,
                    train_y,
                    epochs=50, class_weight=weight,
                    batch_size=batch, verbose=1,
                    validation_data=(val_images, val_y))

results_train = model4.evaluate(train_images, train_y)
results_test = model4.evaluate(test_images, test_y)

print(f'This is the training result: {results_train}')
print(f'This is the test results: {results_test}')

Epoch 1/50
622/622 [==============================] - 47s 74ms/step - loss: 0.5277 - acc: 0.7536 - val_loss: 1.0461 - val_acc: 0.5000
Epoch 2/50
622/622 [==============================] - 45s 73ms/step - loss: 0.4573 - acc: 0.7556 - val_loss: 1.0442 - val_acc: 0.5000
Epoch 3/50
622/622 [==============================] - 45s 73ms/step - loss: 0.4230 - acc: 0.7615 - val_loss: 1.0349 - val_acc: 0.5195
Epoch 4/50
622/622 [==============================] - 46s 75ms/step - loss: 0.3594 - acc: 0.7994 - val_loss: 0.7049 - val_acc: 0.6445
Epoch 5/50
622/622 [==============================] - 46s 74ms/step - loss: 0.3209 - acc: 0.8338 - val_loss: 0.6247 - val_acc: 0.6758
Epoch 6/50
622/622 [==============================] - 47s 75ms/step - loss: 0.2915 - acc: 0.8547 - val_loss: 0.6818 - val_acc: 0.6914
Epoch 7/50
622/622 [==============================] - 47s 75ms/step - loss: 0.2594 - acc: 0.8768 - val_loss: 0.6654 - val_acc: 0.6992
Epoch 8/50
622/622 [==============================] - 47s 76ms

In [ ]:
model5 = models.Sequential()
model5.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3), kernel_regularizer=regularizers.l1(0.005)))
model5.add(layers.Dropout(0.3))

model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(32, (4, 4), activation='relu', kernel_regularizer=regularizers.l1(0.005)))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l1(0.005)))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Flatten())
model5.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.005)))
model5.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l1(0.005)))

model5.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [ ]:
history = model5.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=batch, verbose=1,
                    validation_data=(val_images, val_y))

results_train = model5.evaluate(train_images, train_y)
results_test = model5.evaluate(test_images, test_y)

print(f'This is the training result: {results_train}')
print(f'This is the test results: {results_test}')

Epoch 1/50
622/622 [==============================] - 49s 77ms/step - loss: 1.8217 - acc: 0.7530 - val_loss: 0.9938 - val_acc: 0.5000
Epoch 2/50
622/622 [==============================] - 48s 77ms/step - loss: 0.7093 - acc: 0.7546 - val_loss: 0.9178 - val_acc: 0.5000
Epoch 3/50
622/622 [==============================] - 52s 83ms/step - loss: 0.6945 - acc: 0.7546 - val_loss: 0.9604 - val_acc: 0.5000
Epoch 4/50
622/622 [==============================] - 53s 85ms/step - loss: 0.6882 - acc: 0.7546 - val_loss: 0.9173 - val_acc: 0.5000
Epoch 5/50
622/622 [==============================] - 49s 78ms/step - loss: 0.6808 - acc: 0.7546 - val_loss: 0.9541 - val_acc: 0.5000
Epoch 6/50
622/622 [==============================] - 49s 78ms/step - loss: 0.6752 - acc: 0.7546 - val_loss: 0.9447 - val_acc: 0.5000
Epoch 7/50
622/622 [==============================] - 49s 78ms/step - loss: 0.6736 - acc: 0.7546 - val_loss: 0.9560 - val_acc: 0.5000
Epoch 8/50
622/622 [==============================] - 53s 86ms

In [ ]:
model6 = models.Sequential()
model6.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3), kernel_regularizer=regularizers.l2(0.05)))
model6.add(layers.Dropout(0.3))

model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(32, (4, 4), activation='relu', kernel_regularizer=regularizers.l2(0.05)))
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.05)))
model6.add(layers.MaxPooling2D((2, 2)))
model6.add(layers.Dropout(0.3))

model6.add(layers.Flatten())
model6.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.05)))
model6.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.05)))

model6.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [ ]:
history = model6.fit(train_images,
                    train_y,
                    epochs=50, class_weight=weight,
                    batch_size=batch, verbose=1,
                    validation_data=(val_images, val_y))

results_train = model5.evaluate(train_images, train_y)
results_test = model5.evaluate(test_images, test_y)

print(f'This is the training result: {results_train}')
print(f'This is the test results: {results_test}')

Epoch 1/50
622/622 [==============================] - 49s 77ms/step - loss: 1.8217 - acc: 0.7530 - val_loss: 0.9938 - val_acc: 0.5000
Epoch 2/50
622/622 [==============================] - 48s 77ms/step - loss: 0.7093 - acc: 0.7546 - val_loss: 0.9178 - val_acc: 0.5000
Epoch 3/50
622/622 [==============================] - 52s 83ms/step - loss: 0.6945 - acc: 0.7546 - val_loss: 0.9604 - val_acc: 0.5000
Epoch 4/50
185/622 [=======>......................] - ETA: 41s - loss: 0.6680 - acc: 0.7764